# Notebook 02: Generation Capacity EDA

In this one we'll load in the cleaned data and just play around with it to see what we can learn

In [1]:
import pandas as pd
from src.data.load_data import GenerationCapacity
import altair as alt
import altair_data_server

alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

### Cleaned Data

So you can see now I've cleaned up the generation capacity dataset a bit. If you're interested in the implementation, this is all handled in src/data/load_data.py.

Brief summary of cleaning steps:

* Converted columns with low unique/total ratio into pd.Categorical (good practice rather than storing expensive generic python objects)

* Dropped useless columns like ID, weblink, comment (was empty anyway), capacity_definition (only had one value) etc.

* I noticed there were lots of duplicate entries due to this dataset being a concatenation of multiple data sources. So (although simplistic) I felt the best solution was to drop duplicates on a subset of `['technology', 'year', 'country']` meaning that now each record of those values is unique i.e. there is only 1 record for `['Nuclear', '1998', 'GB']` for example

* Some misc cleaning of values (e.g. converting 'other or unspecified' to 'other' for neatness)

* Converted the year column to a datetime from an integer

* Finally, removed any NaNs that may have been hanging around

In [2]:
clean_cap = GenerationCapacity().load_cleaned()

In [3]:
clean_cap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25298 entries, 0 to 25799
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   technology             25298 non-null  category      
 1   year                   25298 non-null  datetime64[ns]
 2   country                25298 non-null  category      
 3   capacity               25298 non-null  float64       
 4   energy_source_level_0  25298 non-null  int64         
 5   energy_source_level_1  25298 non-null  int64         
 6   energy_source_level_2  25298 non-null  int64         
 7   energy_source_level_3  25298 non-null  int64         
 8   technology_level       25298 non-null  int64         
dtypes: category(2), datetime64[ns](1), float64(1), int64(5)
memory usage: 1.6 MB


In [4]:
clean_cap.sample(10)

,technology,year,country,capacity,energy_source_level_0,energy_source_level_1,energy_source_level_2,energy_source_level_3,technology_level
16612,Renewables,2005-01-01,NO,28900.0000,0,1,0,0,0
11692,Photovoltaics,2010-01-01,IS,0.0000,0,0,0,0,1
12083,Concentrated solar power,2004-01-01,IT,0.0000,0,0,0,0,1
13276,Concentrated solar power,2016-01-01,LU,0.0000,0,0,0,0,1
1686,Differently categorized renewable energy sources,2018-01-01,ES,1025.8405,0,0,1,1,1
4535,Fossil fuels,1993-01-01,CY,0.0000,0,1,0,0,0
23891,Differently categorized hydro,2013-01-01,BE,1430.0000,0,0,0,0,1
4884,Photovoltaics,2011-01-01,CY,10.0000,0,0,0,0,1
10556,Pumped storage,2003-01-01,HU,0.0000,0,0,0,0,1
25703,Non-renewable waste,2018-01-01,SK,19.0000,0,0,1,1,1


In [5]:
clean_cap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25298 entries, 0 to 25799
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   technology             25298 non-null  category      
 1   year                   25298 non-null  datetime64[ns]
 2   country                25298 non-null  category      
 3   capacity               25298 non-null  float64       
 4   energy_source_level_0  25298 non-null  int64         
 5   energy_source_level_1  25298 non-null  int64         
 6   energy_source_level_2  25298 non-null  int64         
 7   energy_source_level_3  25298 non-null  int64         
 8   technology_level       25298 non-null  int64         
dtypes: category(2), datetime64[ns](1), float64(1), int64(5)
memory usage: 1.6 MB


#### Sub-selections

Recall that the `energy_source_level_n` columns refer to how aggregated the data is. With `n=0` being total, `n=1` being split by category, `n=2` being split by individual fuel etc.

This might come in handy to refer to quickly so rather than do a filter every time, I'm going to implement it in the `GenerationCapacity` class.

In [6]:
# Load sub-selection dataframes
totals = GenerationCapacity().load_cleaned(level = 'total')
types = GenerationCapacity().load_cleaned(level = 'type')
fuels = GenerationCapacity().load_cleaned(level = 'fuel')

Let's take a look at a high level how the energy sources differ between countries...

The easiest, high level way of doing this is simply to take an average of the generation capacity by source for each of the countries. Luckily, Altair makes this super easy!

Reason #2875 why Altair is the best python visualisation library!

In [7]:
alt.Chart(data=types).mark_bar().encode(
    x=alt.X("country:N", title="Country", sort="-y"),
    y=alt.Y("mean(capacity):Q", title="Mean Generation Capacity (MW)", stack=True),
    color=alt.Color("technology:N", title="Technology"),
).properties(
    height=500,
    width=750,
    title="Mean Energy Generation Capacity by Source and Country (1990 - 2020)",
)

alt.Chart(...)